# Data acquisition

In [1]:
from pathlib import Path

import pandas as pd
import functions.data_acquisition as data_funcs

%cd ..

import src.configuration as config

Path(config.RAW_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)
Path(config.INTERIM_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)
Path(config.PROCESSED_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)

/workspaces/Madesh9-aviation_final_project


## 1. NTSB incident data

Data source: [NTSB Downloadable Aviation Datasets](https://data.ntsb.gov/avdata)

### 1.1. Download

In [2]:
data_funcs.download_data(config.INCIDENT_DATA_URL, config.RAW_DATA_DIRECTORY, config.RAW_INCIDENTS_MDB_FILE)

### 1.2. Parse and save as CSV

Uses [access_parser](https://github.com/claroty/access_parser) to convert Microsoft Access database file to CSV and then read into Pandas DataFrame.

In [3]:
table=data_funcs.parse_mdb(config.RAW_INCIDENTS_MDB_FILE, config.RAW_INCIDENTS_CSV_FILE)

## 2. On-time performance data

Data source: [Airline Service Quality Performance 234 (On-Time performance data)](https://www.bts.gov/browse-statistical-products-and-data/bts-publications/airline-service-quality-performance-234-time)

### 2.1. Get download links

In [4]:
links=data_funcs.get_ontime_links(config.ONTIME_DATA_URL)

### 2.2. Download on-time data

In [5]:
data_funcs.download_ontime_data(links[:config.ONTIME_FILES], config.ONTIME_DATA_LINK_PREFIX, config.RAW_DATA_DIRECTORY)

### 2.3. Parse and combine on-time datafiles

In [6]:
ontime_df=data_funcs.parse_asc_datafiles(config.ONTIME_FILES, config.RAW_DATA_DIRECTORY, config.RAW_ONTIME_CSV_FILE)

./data/raw/ontime.td.202501 (3).asc
./data/raw/ontime.td.202412.asc


## 3. Feature extraction

### 3.1. Incident data

In [7]:
incidents_df=pd.read_csv(config.RAW_INCIDENTS_CSV_FILE, low_memory=False)
incidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29264 entries, 0 to 29263
Data columns (total 93 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Aircraft_Key             29264 non-null  int64  
 1   cert_max_gr_wt           20043 non-null  float64
 2   fc_seats                 11363 non-null  float64
 3   cc_seats                 1409 non-null   float64
 4   pax_seats                10510 non-null  float64
 5   total_seats              23073 non-null  float64
 6   num_eng                  25043 non-null  float64
 7   date_last_insp           19372 non-null  object 
 8   afm_hrs_last_insp        8004 non-null   float64
 9   afm_hrs                  19004 non-null  float64
 10  dprt_time                18561 non-null  float64
 11  phase_flt_spec           0 non-null      float64
 12  lchg_date                29264 non-null  object 
 13  rwy_len                  12953 non-null  float64
 14  rwy_width             

In [8]:
pd.set_option('display.max_rows', 200)
incidents_df.head().transpose()

0                    1  \
Aircraft_Key                               1                    1   
cert_max_gr_wt                           NaN               2400.0   
fc_seats                                 NaN                  NaN   
cc_seats                                 NaN                  NaN   
pax_seats                                NaN                  NaN   
total_seats                              NaN                  4.0   
num_eng                                  NaN                  1.0   
date_last_insp                           NaN  2007-04-01 00:00:00   
afm_hrs_last_insp                        NaN                 75.0   
afm_hrs                                  NaN               2865.0   
dprt_time                                NaN               2200.0   
phase_flt_spec                           NaN                  NaN   
lchg_date                2023-08-03 13:29:08  2020-09-25 18:05:31   
rwy_len                                  NaN               2700.0   
rwy_width                                NaN                 45.0   
acft_year                                NaN                  NaN   
commercial_space_flight                False                False   
unmanned                               False                False   
ifr_equipped_cert                      False                False   
elt_mounted_aircraft                   False                False   
elt_connected_antenna                  False                False   
ev_id                         20080211X00175       20080107X00026   
regis_no                              N530NA               N8037W   
ntsb_no                           DFW08RA039          SEA08LA057A   
acft_missing                               N                    N   
far_part                                NUSN                  091   
flt_plan_filed                           NaN                 NONE   
flight_plan_activated                    NaN                  NaN   
damage                                  SUBS                 MINR   
acft_fire                                NaN                 NONE   
acft_expl                                NaN                 NONE   
acft_make                  Eurocopter France                PIPER   
acft_model                           AS350B3            PA 28-180   
acft_series                              NaN                  NaN   
acft_serial_no                           NaN              28-2104   
acft_category                           HELI                  AIR   
acft_reg_cls                             NaN                  NaN   
homebuilt                                  N                    N   
fixed_retractable                       FIXD                 FIXD   
type_last_insp                           NaN                 ANNL   
elt_install                              NaN                    Y   
elt_oper                                 NaN                    N   
elt_aided_loc_ev                         NaN                    N   
elt_type                                 NaN                  NaN   
owner_acft                               NaN       Marshall White   
owner_street                             NaN                  NaN   
owner_city                               NaN           Sebastopol   
owner_state                              NaN                   CA   
owner_country                            NaN                  USA   
owner_zip                                NaN                95472   
oper_individual_name                       N                    N   
oper_name                  Prism Helicopters      Timothy Bennett   
oper_same                                NaN                  NaN   
oper_dba                                 NaN                  NaN   
oper_addr_same                           NaN                  NaN   
oper_street                              NaN                  NaN   
oper_city                                NaN               Novato   
oper_state                               

In [9]:
incident_features={
    'dprt_time': 'departure_time',
    'dprt_apt_id': 'origin',
    'dest_apt_id': 'destination',
    'regis_no': 'tail_number'
}

extracted_incident_df=incidents_df[incident_features.keys()].copy()
extracted_incident_df=extracted_incident_df.rename(columns=incident_features)
extracted_incident_df['incident']=[1]*len(extracted_incident_df)
extracted_incident_df.dropna(inplace=True)
extracted_incident_df.to_csv(config.EXTRACTED_INCIDENTS_FILE, index=False)
extracted_incident_df.head()

departure_time origin destination tail_number  incident
1          2200.0   O69         OQ3       N8037W         1
2          2230.0   OQ3         O70        N15EX         1
3          2100.0   T20         AXH       N579RM         1
4           400.0   PHX         PWA        N398J         1
5          2215.0   KFNL        KFNL      N24965         1

In [10]:
extracted_incident_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14413 entries, 1 to 29225
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   departure_time  14413 non-null  float64
 1   origin          14413 non-null  object 
 2   destination     14413 non-null  object 
 3   tail_number     14413 non-null  object 
 4   incident        14413 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 675.6+ KB


### 3.2. On-time data

In [11]:
ontime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231062 entries, 0 to 1231061
Data columns (total 96 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   DL           1231062 non-null  object 
 1   5030         599028 non-null   float64
 2   Unnamed: 2   41 non-null       object 
 3   Unnamed: 3   41 non-null       float64
 4   9E           1231062 non-null  object 
 5   5030.1       599028 non-null   float64
 6   CVG          599028 non-null   object 
 7   LGA          1231062 non-null  object 
 8   20250127     599028 non-null   float64
 9   1            599028 non-null   float64
 10  1700         599028 non-null   float64
 11  1700.1       599028 non-null   float64
 12  1658         599028 non-null   float64
 13  1859         599028 non-null   float64
 14  1859.1       599028 non-null   float64
 15  1902         599028 non-null   float64
 16  0            1231062 non-null  int64  
 17  0.1          1231062 non-null  int64  
 18  11

In [12]:
ontime_df.head().transpose()

0           1           2           3           4
DL                   DL          DL          DL          DL          DL
5030             5030.0      5030.0      5030.0      5030.0      5030.0
Unnamed: 2          NaN         NaN         NaN         NaN         NaN
Unnamed: 3          NaN         NaN         NaN         NaN         NaN
9E                   9E          9E          9E          9E          9E
5030.1           5030.0      5030.0      5030.0      5030.0      5030.0
CVG                 CVG         CVG         CVG         CVG         LGA
LGA                 LGA         LGA         LGA         LGA         CVG
20250127     20250128.0  20250129.0  20250130.0  20250131.0  20250102.0
1                   2.0         3.0         4.0         5.0         4.0
1700             1700.0      1700.0      1700.0      1700.0      1335.0
1700.1           1700.0      1700.0      1700.0      1700.0      1335.0
1658             1647.0      1719.0      1656.0      1650.0      1328.0
1859             1859.0      1859.0      1859.0      1859.0      1558.0
1859.1           1859.0      1859.0      1859.0      1859.0      1558.0
1902             1829.0      1905.0      1831.0      1851.0      1537.0
0                     0           0           0           0           0
0.1                   0           0           0           0           0
119               119.0       119.0       119.0       119.0       143.0
124               102.0       106.0        95.0       121.0       129.0
-2                -13.0        19.0        -4.0       -10.0        -7.0
3                 -30.0         6.0       -28.0        -8.0       -21.0
5                   -17         -13         -24           2         -14
1716             1702.0      1738.0      1706.0      1710.0      1345.0
1853             1824.0      1902.0      1825.0      1843.0      1532.0
N917XJ           N914XJ      N478PX      N917XJ      N914XJ      N166PQ
18                 15.0        19.0        10.0        20.0        17.0
9                   5.0         3.0         6.0         8.0         5.0
97                 82.0        84.0        79.0        93.0       107.0
Unnamed: 29         NaN         NaN         NaN         NaN         NaN
0.2                   0           0           0           0           0
0.3                   0           0           0           0           0
0.4                   0           0           0           0           0
0.5                   0           0           0           0           0
0.6                   0           0           0           0           0
0.7                   0           0           0           0           0
0.8                   0           0           0           0           0
0.9                   0           0           0           0           0
0.10                  0           0           0           0           0
Unnamed: 39         NaN         NaN         NaN         NaN         NaN
0.11                  0           0           0           0           0
0.12                  0           0           0           0           0
0.13                  0           0           0           0           0
0.14                  0           0           0           0           0
Unnamed: 44         NaN         NaN         NaN         NaN         NaN
Unnamed: 45         NaN         NaN         NaN         NaN         NaN
0.15                  0           0           0           0           0
0.16                  0           0           0           0           0
0.17                  0           0           0           0           0
0.18                  0           0           0           0           0
Unnamed: 50         NaN         NaN         NaN         NaN         NaN
Unnamed: 51         NaN         NaN         NaN         NaN         NaN
0.19                  0           0           0           0           0
0.20                  0           0           0           0           0
0.21                  0           0           0           0   

In [13]:
ontime_features={
    # 'carrier': 0,
    # 'flight_number': 1,
    'origin': 6,
    'destination': 7,
    # 'date': 8,
    'departure_time': 12,
    'tail_number': 25 
}

extracted_ontime_df=ontime_df.iloc[:,list(ontime_features.values())].copy()
extracted_ontime_df.columns=ontime_features.keys()
extracted_ontime_df['incident']=[0]*len(extracted_ontime_df)
extracted_ontime_df.dropna(inplace=True)
extracted_ontime_df.to_csv(config.EXTRACTED_ONTIME_FILE, index=False)
extracted_ontime_df.head()

origin destination  departure_time tail_number  incident
0    CVG         LGA          1647.0      N914XJ         0
1    CVG         LGA          1719.0      N478PX         0
2    CVG         LGA          1656.0      N917XJ         0
3    CVG         LGA          1650.0      N914XJ         0
4    LGA         CVG          1328.0      N166PQ         0

In [14]:
extracted_ontime_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 595162 entries, 0 to 599027
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   origin          595162 non-null  object 
 1   destination     595162 non-null  object 
 2   departure_time  595162 non-null  float64
 3   tail_number     595162 non-null  object 
 4   incident        595162 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 27.2+ MB


### 3.3. Combine incident and on-time data

In [15]:
data_df=pd.concat([extracted_ontime_df, extracted_incident_df], axis=0)
data_df.reset_index(inplace=True, drop=True)
data_df['origin']=data_df['origin'].astype(str)
data_df['destination']=data_df['destination'].astype(str)
data_df['departure_time']=data_df['departure_time'].astype(float)
data_df['tail_number']=data_df['tail_number'].astype(str)
data_df['incident']=data_df['incident'].astype(int)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609575 entries, 0 to 609574
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   origin          609575 non-null  object 
 1   destination     609575 non-null  object 
 2   departure_time  609575 non-null  float64
 3   tail_number     609575 non-null  object 
 4   incident        609575 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 23.3+ MB


### 3.4. Save

In [16]:
data_df.to_csv(config.COMBINED_DATAFILE, index=False)